### This model was trained on the first set of limited data- 246 train and 38 test examples 
<br>
> - The procedure used for loading data here is different from elsewhere. This was the initial method after which i wrote a better one. In this, 'files_train.csv' contains the names of all files. These are used to load data. 
<br>
<br>
> - This model has a validation accuracy of 71%

In [1]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading training data names into train_files
fin = open("files_train.csv", 'r')
train_files = []
for line in fin:
    train_files.append(line)
print(len(train_files))

#loading test data names into test_files
fin = open("files_test.csv", 'r')
test_files = []
for line in fin:
    test_files.append(line)
print(len(test_files))

246
38


In [3]:
# load training images 
train_images = []
train_y = []
for i in range(len(train_files)):
    data_path = "dog_images_train/"
    img = load_img(path=data_path+(train_files[i].strip()), target_size=(256,256))
    img = img_to_array(img)
    train_images.append(img)
    if train_files[i].startswith('dog'):
        train_y.append(1) 
    else:
        train_y.append(0)
train_images = np.array(train_images)
print(train_images.shape)

(246, 256, 256, 3)


In [4]:
# load testing images 
test_images = []
test_y = []
for i in range(len(test_files)):
    data_path = "dog_images_test/"
    img = load_img(path=data_path+(test_files[i].strip()), target_size=(256,256))
    img = img_to_array(img)
    test_images.append(img)
    if test_files[i].startswith('dog'):
        test_y.append(1) 
    else:
        test_y.append(0)

test_images = np.array(test_images)
print(test_images.shape)

(38, 256, 256, 3)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [12]:
batch_size = 16
train_datagen = ImageDataGenerator(
        rotation_range=26,
        width_shift_range=0.15,
        height_shift_range=0.15,
        rescale=1./255,
        shear_range=0.17,
        zoom_range=0.16,
        horizontal_flip=True,
        fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train_generator = train_datagen.flow(train_images, train_y, batch_size=batch_size)
validation_generator = test_datagen.flow(test_images, test_y)
model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, epochs = 50, validation_data=validation_generator)


Epoch 1/50
15/15 [==============================] - 4s 267ms/step - loss: 0.3695 - acc: 0.8458 - val_loss: 0.8368 - val_acc: 0.6579
Epoch 2/50
15/15 [==============================] - 4s 240ms/step - loss: 0.5061 - acc: 0.7961 - val_loss: 0.6031 - val_acc: 0.7368
Epoch 3/50
15/15 [==============================] - 4s 251ms/step - loss: 0.4495 - acc: 0.8500 - val_loss: 0.7012 - val_acc: 0.6316
Epoch 4/50
15/15 [==============================] - 4s 235ms/step - loss: 0.4190 - acc: 0.8236 - val_loss: 0.7498 - val_acc: 0.6842
Epoch 5/50
15/15 [==============================] - 4s 243ms/step - loss: 0.3653 - acc: 0.8324 - val_loss: 0.7513 - val_acc: 0.6579
Epoch 6/50
15/15 [==============================] - 4s 245ms/step - loss: 0.3001 - acc: 0.8640 - val_loss: 1.0855 - val_acc: 0.7105
Epoch 7/50
15/15 [==============================] - 4s 255ms/step - loss: 0.5206 - acc: 0.7750 - val_loss: 0.7215 - val_acc: 0.6842
Epoch 8/50
15/15 [==============================] - 3s 232ms/step - loss: 0.

In [14]:
model.save_weights('first_try.h5')